In [5]:
!pip install datasets

In [ ]:
!pip install sympy --upgrade

   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.2 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.2 MB 699.0 kB/s eta 0:00:09
   --- ------------------------------------ 0.5/6.2 MB 699.0 kB/s eta 0:00:09
   ----- ---------------------------------- 0.8/6.2 MB 817.9 kB/s eta 0:00:07
   ------ --------------------------------- 1.0/6.2 MB 786.4 kB/s eta 0:00:07
   ------ --------------------------------- 1.0/6.2 MB 786.4 kB/s eta 0:00:07
   -------- ------------------------------- 1.3/6.2 MB 762.6 kB/s eta 0:00:07
   -------- ------------------------------- 1.3/6.2 MB 762.6 kB/s eta 0:00:07
   ---------- ----------------------------- 1.6/6.2 MB 755.9 kB/s eta 0:00:07
   ---------- ----------------------------- 1.6/6.2 MB 755.9 kB/s eta 0:00:07
   ----------- -------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.


In [ ]:
!pip install torch torchvision timm opencv-python numpy scikit-learn pillow


  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3


In [ ]:
!pip install opencv-python


In [ ]:
from sklearn import datasets
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
import os
from torchvision.datasets import ImageFolder
import torch.nn as nn
import timm
from torchvision import models
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

# Set dataset path
dataset_path = "/Deepfake/code/Dataset/"
train_dataset_path = os.path.join(dataset_path, "Train")
test_dataset_path = os.path.join(dataset_path, "Test")



# Load datasets
full_dataset = ImageFolder(root=train_dataset_path)
test_dataset = ImageFolder(root=test_dataset_path)

# Define transformations
IMG_SIZE = 224
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Apply transforms
full_dataset.transform = train_transforms
test_dataset.transform = val_transforms

# Split dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms

BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Model Architecture
class HybridDeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.effnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        self.effnet.classifier = nn.Identity()
        self.vit = timm.create_model("vit_base_patch16_224", pretrained=True)
        self.vit.head = nn.Identity()
        self.classifier = nn.Sequential(
            nn.Linear(2048, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        eff_features = self.effnet(x)
        vit_features = self.vit(x)
        combined_features = torch.cat([eff_features, vit_features], dim=1)
        return self.classifier(combined_features)

# Initialize model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridDeepfakeDetector().to(DEVICE)

# Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2)
scaler = torch.cuda.amp.GradScaler()

# Training Loop
EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.float().to(DEVICE).unsqueeze(1)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(True):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * images.size(0)
        torch.cuda.empty_cache()
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss / len(train_loader.dataset):.4f}")
    scheduler.step(train_loss)

# Model Evaluation

def evaluate_model(model, test_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, np.round(predictions))
    auc = roc_auc_score(true_labels, predictions)
    f1 = f1_score(true_labels, np.round(predictions))
    precision = precision_score(true_labels, np.round(predictions))
    recall = recall_score(true_labels, np.round(predictions))
    return accuracy, auc, f1, precision, recall

accuracy, auc, f1, precision, recall = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")

# Single Image Prediction
def predict(image_path, model):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = val_transforms(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        output = model(image)
        prediction = torch.sigmoid(output).item()
    return "Fake" if prediction > 0.5 else "Real"

# Example usage
print(predict("/content/testimage deepfake.jpg", model))


Training samples: 1625
Validation samples: 407
Test samples: 2064


C:\Users\A\AppData\Local\Temp\ipykernel_8532\2455564840.py:90: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
C:\Users\A\AppData\Local\Temp\ipykernel_8532\2455564840.py:100: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):


Epoch 1/20 - Train Loss: 0.3999
Epoch 2/20 - Train Loss: 0.1831
Epoch 3/20 - Train Loss: 0.1070
Epoch 4/20 - Train Loss: 0.0880
Epoch 5/20 - Train Loss: 0.0504
Epoch 6/20 - Train Loss: 0.0575
Epoch 7/20 - Train Loss: 0.0503
Epoch 8/20 - Train Loss: 0.0353
Epoch 9/20 - Train Loss: 0.0292
Epoch 10/20 - Train Loss: 0.0158
Epoch 11/20 - Train Loss: 0.0112
Epoch 12/20 - Train Loss: 0.0333
Epoch 13/20 - Train Loss: 0.0263
Epoch 14/20 - Train Loss: 0.0189
Epoch 15/20 - Train Loss: 0.0083
Epoch 16/20 - Train Loss: 0.0158
Epoch 17/20 - Train Loss: 0.0080
Epoch 18/20 - Train Loss: 0.0045
Epoch 19/20 - Train Loss: 0.0090
Epoch 20/20 - Train Loss: 0.0060
Test Accuracy: 0.8774
Test AUC: 0.9351
Test F1 Score: 0.8863
Test Precision: 0.8435
Test Recall: 0.9337


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\content\\testimage deepfake.jpg'

In [ ]:
from sklearn import datasets
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
import os
from torchvision.datasets import ImageFolder
import torch.nn as nn
import timm
from torchvision import models
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

# Set dataset path
dataset_path = "/Deepfake/code/FF++/"
train_dataset_path = os.path.join(dataset_path, "Train")
test_dataset_path = os.path.join(dataset_path, "Test")

# Load datasets
full_dataset = ImageFolder(root=train_dataset_path)
test_dataset = ImageFolder(root=test_dataset_path)

# Define transformations
IMG_SIZE = 224
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Apply transforms
full_dataset.transform = train_transforms
test_dataset.transform = val_transforms

# Split dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms

BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Model Architecture with EfficientNet-B1
class HybridDeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.effnet = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.DEFAULT)
        self.effnet.classifier = nn.Identity()
        self.vit = timm.create_model("vit_base_patch16_224", pretrained=True)
        self.vit.head = nn.Identity()
        self.classifier = nn.Sequential(
        nn.Linear(2048, 128),
        nn.LeakyReLU(negative_slope=0.01),
        nn.Dropout(0.3),
        nn.Linear(128, 64),
        nn.Linear(64, 1)
        )

    def forward(self, x):
        eff_features = self.effnet(x)
        vit_features = self.vit(x)
        combined_features = torch.cat([eff_features, vit_features], dim=1)
        return self.classifier(combined_features)

# Initialize model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridDeepfakeDetector().to(DEVICE)

# Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2)
scaler = torch.cuda.amp.GradScaler()

# Training Loop
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.float().to(DEVICE).unsqueeze(1)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(True):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * images.size(0)
        torch.cuda.empty_cache()
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss / len(train_loader.dataset):.4f}")
    scheduler.step(train_loss)

# Model Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, np.round(predictions))
    auc = roc_auc_score(true_labels, predictions)
    f1 = f1_score(true_labels, np.round(predictions))
    precision = precision_score(true_labels, np.round(predictions))
    recall = recall_score(true_labels, np.round(predictions))
    return accuracy, auc, f1, precision, recall

accuracy, auc, f1, precision, recall = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")

# Single Image Prediction
def predict(image_path, model):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = val_transforms(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        output = model(image)
        prediction = torch.sigmoid(output).item()
    return "Fake" if prediction > 0.5 else "Real"

# Example usage
print(predict("/content/testimage deepfake.jpg", model))

#saving 
checkpoint_path = "checkpoint_epoch_19.pth"  # update if using different file
start_epoch = 20
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resumed training from epoch {start_epoch}")
else:
    print("No checkpoint found. Starting fresh training.")

        # ✅ Save checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
    }, f'checkpoint_epoch_{epoch}.pth')

KeyboardInterrupt: 

In [3]:
from sklearn import datasets
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
import os
from torchvision.datasets import ImageFolder
import torch.nn as nn
import timm
from torchvision import models
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

# Set dataset path
dataset_path = "/Deepfake/code/Dataset/"
train_dataset_path = os.path.join(dataset_path, "Train")
test_dataset_path = os.path.join(dataset_path, "Test")

# Load datasets
full_dataset = ImageFolder(root=train_dataset_path)
test_dataset = ImageFolder(root=test_dataset_path)

# Define transformations
IMG_SIZE = 224
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Apply transforms
full_dataset.transform = train_transforms
test_dataset.transform = val_transforms

# Split dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms

BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Model Architecture with EfficientNet-B1 and ViT
class HybridDeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.effnet = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.DEFAULT)
        self.effnet.classifier = nn.Identity()
        self.vit = timm.create_model("vit_base_patch16_224", pretrained=True)
        self.vit.head = nn.Identity()
        self.classifier = nn.Sequential(
            nn.Linear(2048, 128),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        eff_features = self.effnet(x)
        vit_features = self.vit(x)
        combined_features = torch.cat([eff_features, vit_features], dim=1)
        return self.classifier(combined_features)

# Device, model, loss, optimizer
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridDeepfakeDetector().to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2)
scaler = torch.cuda.amp.GradScaler()

# Resume training if checkpoint exists
checkpoint_path = "checkpoint_epoch_19.pth"
start_epoch = 0
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resumed training from epoch {start_epoch}")
else:
    print("No checkpoint found. Starting fresh training.")

# Training Loop
EPOCHS = 10
for epoch in range(start_epoch, EPOCHS):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.float().to(DEVICE).unsqueeze(1)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast("cuda"):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * images.size(0)
        torch.cuda.empty_cache()
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss / len(train_loader.dataset):.4f}")
    scheduler.step(train_loss)

    # Save checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
    }, f'checkpoint_epoch_{epoch}.pth')

# Model Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, np.round(predictions))
    auc = roc_auc_score(true_labels, predictions)
    f1 = f1_score(true_labels, np.round(predictions))
    precision = precision_score(true_labels, np.round(predictions))
    recall = recall_score(true_labels, np.round(predictions))
    return accuracy, auc, f1, precision, recall

accuracy, auc, f1, precision, recall = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")

# Predict single image
#def predict(image_path, model):
    #model.eval()
   # image = Image.open(image_path).convert("RGB")
   # image = val_transforms(image).unsqueeze(0).to(DEVICE)
   # with torch.no_grad():
   #     output = model(image)
   #     prediction = torch.sigmoid(output).item()
   # return "Fake" if prediction > 0.5 else "Real"

# Example single image
#print(predict("/content/testimage_deepfake.jpg", model))

# Predict multiple images
def predict_multiple(images_dir, model):
    model.eval()
    results = {}
    for filename in os.listdir(images_dir):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(images_dir, filename)
            try:
                image = Image.open(image_path).convert("RGB")
                image = val_transforms(image).unsqueeze(0).to(DEVICE)
                with torch.no_grad():
                    output = model(image)
                    prediction = torch.sigmoid(output).item()
                label = "Fake" if prediction > 0.5 else "Real"
                results[filename] = (label, round(prediction, 4))
            except Exception as e:
                results[filename] = f"Error: {e}"
    return results

# Example multiple image usage
image_folder_path = "/content/multiple_test_images/"
predictions = predict_multiple(image_folder_path, model)
for image_name, result in predictions.items():
    print(f"{image_name}: {result}")


Training samples: 1625
Validation samples: 407
Test samples: 2064


C:\Users\A\AppData\Local\Temp\ipykernel_21432\2722016111.py:87: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
C:\Users\A\AppData\Local\Temp\ipykernel_21432\2722016111.py:110: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast("cuda"):


No checkpoint found. Starting fresh training.
Epoch 1/10 - Train Loss: 0.4106
Epoch 2/10 - Train Loss: 0.1652
Epoch 3/10 - Train Loss: 0.1071
Epoch 4/10 - Train Loss: 0.0604
Epoch 5/10 - Train Loss: 0.0501
Epoch 6/10 - Train Loss: 0.0458
Epoch 7/10 - Train Loss: 0.0343
Epoch 8/10 - Train Loss: 0.0376
Epoch 9/10 - Train Loss: 0.0156
Epoch 10/10 - Train Loss: 0.0099
Test Accuracy: 0.8643
Test AUC: 0.9431
Test F1 Score: 0.8756
Test Precision: 0.8250
Test Recall: 0.9328


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/multiple_test_images/'